# Hedging a 3-Leg Parlay Bet in Prediction Markets




In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd

# Set display options
pd.set_option('display.precision', 4)

## Introduction

In this notebook, we will explore how a Liquidity Provider (LP) can hedge a 3-leg parlay bet in a prediction market. The goal is to:

- Limit the LP's potential losses.
- Reflect parlay bets in market liquidity.
- Achieve expected profitability by applying a margin to the odds.

## Parlay Bet Details

- Number of Legs: 3 events
- Probability of Each Event: 50% (independent events)
- User's Wager: $1
- Margin Applied: We will apply a margin to adjust the odds.

In [20]:
# True probability of each event
p_event = 0.5

# True odds for each event
odds_true = 1 / p_event
print(f"True odds for each event: {odds_true:.2f}x")


True odds for each event: 2.00x


In [21]:
# Combined probability of winning the parlay
p_parlay = p_event ** 3  # Since events are independent

# Combined true odds for the parlay
odds_parlay_true = 1 / p_parlay
print(f"Combined true odds for the parlay: {odds_parlay_true:.2f}x")


Combined true odds for the parlay: 8.00x


## Adjusting Odds with Margin

We will apply a margin (overround) to the odds to ensure expected profitability.
Applying Margin to Individual Event Odds

Let's assume a margin of 5%.

In [84]:
# Margin to apply (5%)
margin = 0.05

# Adjusted probability for each event
p_event_adjusted = p_event * (1 + margin)

# Adjusted odds for each event
odds_event_adjusted = 1 / p_event_adjusted
print(f"Adjusted odds for each event: {odds_event_adjusted:.4f}x")


Adjusted odds for each event: 1.9048x


In [85]:
# Combined adjusted odds for the parlay
odds_parlay_adjusted = odds_event_adjusted ** 3
print(f"Adjusted odds for the parlay: {odds_parlay_adjusted:.4f}x")


Adjusted odds for the parlay: 6.9107x


In [86]:
# User's potential payout
payout_user = odds_parlay_adjusted * 1  # Wager is $1
print(f"Potential payout to user: ${payout_user:.4f}")


Potential payout to user: $6.9107


## LP's Hedging Strategy

The LP hedges by placing bets on the same outcomes in the underlying markets at true odds.


## Possible Outcome Scenarios

There are 8 possible outcomes for the three events.
Generating 

### All Possible Outcomes

In [87]:
# Generate all combinations of outcomes (Y for YES, N for NO)
outcomes = ['Y', 'N']
all_scenarios = pd.DataFrame(
    [(a, b, c) for a in outcomes for b in outcomes for c in outcomes],
    columns=['Event 1', 'Event 2', 'Event 3']
)

# Add scenario probabilities (each scenario is equally likely)
all_scenarios['Probability'] = 1 / 8  # 8 total scenarios

# Display the scenarios
all_scenarios


,Event 1,Event 2,Event 3,Probability
0,Y,Y,Y,0.125
1,Y,Y,N,0.125
2,Y,N,Y,0.125
3,Y,N,N,0.125
4,N,Y,Y,0.125
5,N,Y,N,0.125
6,N,N,Y,0.125
7,N,N,N,0.125


In [88]:
# Initialize columns for calculations
all_scenarios['User Wins Parlay'] = False
all_scenarios['LP Pays User'] = 0.0
all_scenarios['LP Receives from User'] = 1.0  # User's wager
all_scenarios['LP Hedging Gain/Loss'] = 0.0
all_scenarios['LP Net Profit/Loss'] = 0.0

# Hedging amount per event
hedge_bet_per_event = 1  # $1 on each event

# Total hedging cost
hedge_total_cost = hedge_bet_per_event * 3
print(f"Total hedging cost: ${hedge_total_cost:.2f}")

# Loop through each scenario to calculate outcomes
for idx, row in all_scenarios.iterrows():
    events = [row['Event 1'], row['Event 2'], row['Event 3']]
    
    # Check if user wins the parlay (all events YES)
    if events == ['Y', 'Y', 'Y']:
        all_scenarios.at[idx, 'User Wins Parlay'] = True
        all_scenarios.at[idx, 'LP Pays User'] = payout_user  # LP pays the adjusted payout
    
    # Calculate LP's hedging gain/loss
    hedging_gain = 0.0
    for event in events:
        if event == 'Y':
            # LP wins hedging bet (bet on YES at true odds)
            net_gain = (2 * hedge_bet_per_event) - hedge_bet_per_event  # Net gain per event
            hedging_gain += net_gain
        else:
            # LP loses hedging bet
            hedging_gain -= hedge_bet_per_event  # Net loss per event
    all_scenarios.at[idx, 'LP Hedging Gain/Loss'] = hedging_gain
    
    # Calculate LP's net profit/loss
    lp_inflows = all_scenarios.at[idx, 'LP Receives from User'] + all_scenarios.at[idx, 'LP Hedging Gain/Loss']
    lp_outflows = all_scenarios.at[idx, 'LP Pays User']  # Remove hedge_total_cost
    all_scenarios.at[idx, 'LP Net Profit/Loss'] = lp_inflows - lp_outflows

# Display the updated scenarios with calculations
print(all_scenarios[['Event 1', 'Event 2', 'Event 3', 'User Wins Parlay', 'LP Net Profit/Loss']])


Total hedging cost: $3.00
  Event 1 Event 2 Event 3  User Wins Parlay  LP Net Profit/Loss
0       Y       Y       Y              True             -2.9107
1       Y       Y       N             False              2.0000
2       Y       N       Y             False              2.0000
3       Y       N       N             False              0.0000
4       N       Y       Y             False              2.0000
5       N       Y       N             False              0.0000
6       N       N       Y             False              0.0000
7       N       N       N             False             -2.0000


### Calculating LP's Net Profit/Loss in Each Scenario

Calculating Expected Profit

In [89]:
# Calculate expected profit
all_scenarios['Expected Profit'] = all_scenarios['Probability'] * all_scenarios['LP Net Profit/Loss']
expected_profit = all_scenarios['Expected Profit'].sum()
print(f"LP's Expected Profit per $1 wagered: ${expected_profit:.4f}")


LP's Expected Profit per $1 wagered: $0.1362


## Capital Requirements

### Maximum Potential Loss

In [90]:
# Maximum potential loss
max_loss = all_scenarios['LP Net Profit/Loss'].min()
print(f"Maximum potential loss for LP per $1 wagered: ${-max_loss:.4f}")


Maximum potential loss for LP per $1 wagered: $2.9107
